In [1]:
import pandas as pd
import itertools
from prophet import Prophet
import logging
import cmdstanpy
import pickle
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_4 = pd.read_csv('data_for_model.csv')
data_4['date'] = pd.to_datetime(data_4['date'])

In [3]:
cities_list = list(data_4['city'].drop_duplicates())
len(cities_list)

109

In [4]:
russian_cities = list(data_4[data_4['country']=='Russia']['city'].drop_duplicates())
russian_cities.remove('Moscow')
russian_cities.remove('Saint Petersburg')
len(russian_cities)

9

In [5]:
cities_list = [x for x in cities_list if x not in russian_cities]
#cities_list = list(set(cities_list) - set(russian_cities))
len(cities_list)

100

In [6]:
logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.WARNING)
#logger.setLevel(logging.INFO)

In [7]:
data_4[data_4['Price per Square Meter to Buy Apartment Outside of Centre']>20000][['city','date','Price per Square Meter to Buy Apartment in City Centre','Price per Square Meter to Buy Apartment Outside of Centre']]

,city,date,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre
672,Brno,2010-03-31,52872.752908,39654.564681
673,Brno,2010-06-30,52872.752908,39654.564681
674,Brno,2010-09-30,52872.752908,39654.564681
675,Brno,2010-12-31,52872.752908,39654.564681
676,Brno,2011-03-31,52872.752908,39654.564681
677,Brno,2011-06-30,52872.752908,39654.564681
678,Brno,2011-09-30,52872.752908,39654.564681
679,Brno,2011-12-31,52872.752908,39654.564681
3256,Moscow,2012-03-31,166043.569355,89295.996710
3257,Moscow,2012-06-30,166043.569355,89295.996710


In [8]:
# Moscow and Saint Petersburg have abnormally high prices in 2012, Prague in 2011 and Brno in 2010 and 2011.
# Replacing with the average: 

for col in ['Price per Square Meter to Buy Apartment in City Centre','Price per Square Meter to Buy Apartment Outside of Centre']:
    for city in ['Moscow', 'Saint Petersburg']:
        average_2011_2013 = data_4[(data_4['city'] == city) & 
                                           ((data_4['date'].dt.year == 2011) | (data_4['date'].dt.year == 2013))][col]

        average_value = average_2011_2013.mean()

        data_4.loc[(data_4['city'] == city) & (data_4['date'].dt.year == 2012),
                           col] = average_value

    for city in ['Prague']:
        average_2010_2012 = data_4[(data_4['city'] == city) & 
                                           ((data_4['date'].dt.year == 2010) | (data_4['date'].dt.year == 2012))][col]

        average_value = average_2010_2012.mean()

        data_4.loc[(data_4['city'] == city) & (data_4['date'].dt.year == 2011),
                           col] = average_value
        
    for city in ['Brno']:
        average_2012 = data_4[(data_4['city'] == city) & (data_4['date'].dt.year == 2012)][col]

        average_value = average_2012.mean()

        data_4.loc[(data_4['city'] == city) & (data_4['date'].dt.year <= 2011),
                           col] = average_value

In [5]:
# Finding the models for each city with the best combination of feature variables

#cities_list = cities_list[0:5]
features_model_TOTAL = []
cities_TOTAL = []
num_feats = []
error_list = []
data_5 = data_4.copy()
total_predictions = pd.DataFrame()
target_var = 'Price per Square Meter to Buy Apartment in City Centre'
location = 'centre' # for pickle files

for city in cities_list:
    print('---',cities_list.index(city)+1,city)
    # City data
    data_city = data_5[data_5['city'] == city]
    data_city = data_city[(data_city['date']>='2011-01-01')]
    clean_target_var = target_var.replace(' ', '_').replace('(', '').replace(')', '')
    data_city = data_city.rename(columns={'date': 'ds'})
    
    # Target city data
    data_city_target = data_city[['ds',target_var]].reset_index(drop=True)
    data_city_target.columns = ['ds','y']
    
    # Features city data
    cols_to_remove = ['city','country','year','Price per Square Meter to Buy Apartment in City Centre',
                     'Price per Square Meter to Buy Apartment Outside of Centre']
    data_city_features = data_city.drop(columns=cols_to_remove)
    
    # (Re)Setting variables for city
    features_model = []
    var_min = 1
    current_min = 0.1

    # Finding best model for city
    while current_min <= var_min and current_min > 0:
        feature_colls = list(data_city_features.drop(columns=['ds']).columns)

        for feat in features_model:
            feature_colls.remove(feat)

        features_list_t = []
        abs_var_list_t = []
        
        # Finding best variables for the model
        for feature in feature_colls:
            feature_data_temp = data_city_features[['ds',feature] + features_model]
            target_data_temp = data_city_target[data_city_target['ds']<='2020-12-31']
            past_feature_data_temp = feature_data_temp[feature_data_temp['ds']<='2020-12-31']
            future_feature_data_temp = feature_data_temp[(feature_data_temp['ds']>'2020-12-31') & 
                                                         (feature_data_temp['ds']<'2023-01-01')]

            m_temp = Prophet()
            for feat in features_model:
                m_temp.add_regressor(feat)
            m_temp.add_regressor(feature)
            logger.setLevel(logging.WARNING)
            m_temp.fit(pd.merge(target_data_temp, past_feature_data_temp, on='ds'))


            future_temp = m_temp.make_future_dataframe(periods=8, freq='Q') # creating test date columns (2021 and 2022)
            future_temp = pd.merge(future_temp, feature_data_temp, on='ds')

            forecast_temp = m_temp.predict(future_temp)
            df_temp = pd.merge(forecast_temp[['ds','yhat']], data_city_target, on='ds').tail(8) # 2021 and 2022 (8 quarters)
            df_temp['abs_var'] = abs((df_temp['yhat']-df_temp['y'])/df_temp['y']) 
            var = df_temp['abs_var'].sum()/8

            features_list_t.append(feature)
            abs_var_list_t.append(var)

        # Creating dataframe with additional feature model's overall absolute variation
        df_dif = pd.DataFrame({'feature':features_list_t,'abs_var':abs_var_list_t})
        df_dif = df_dif.sort_values(by='abs_var')
        df_dif = df_dif.reset_index(drop=True)
        current_min = df_dif['abs_var'].min()
        min_feat = df_dif.loc[df_dif['abs_var'].idxmin(), 'feature'] # selecting best feature

        if df_dif['abs_var'].sum() == 0: # needed to stop loop for when best model has no features and abs vars are 0
            var_min = 0
            overall_min = 0
        else:
            pass
        
        if current_min < var_min:
            features_model.append(min_feat) # adding best feature to list of variables for city model
            features_model_TOTAL.append(min_feat) # adding best feature to list of variables for all cities models
            cities_TOTAL.append(city) # adding city to create dataframe for feats list
            df_dif_final = df_dif.copy()
            #print('current_min:', current_min,'var_min', var_min)
            #print(features_model)
            var_min = current_min
            overall_min = current_min
        else:
            var_min = 0
    
    error_list.append(overall_min)
    num_feats.append(len(features_model))
    print(len(features_model), 'features selected for', city, 'with error margin of', round(overall_min*100,2),'%',':'
          , features_model)

    # Final city model and preditions for 2023
    feature_data_final = data_city_features[['ds'] + features_model]
    target_data_final = data_city_target[data_city_target['ds']<='2020-12-31']
    past_feature_data_final = feature_data_final[feature_data_final['ds']<='2020-12-31']
    future_feature_data_final = feature_data_final[feature_data_final['ds']>'2020-12-31']

    m_final = Prophet()
    for feat in features_model:
        m_final.add_regressor(feat)

    m_final.fit(pd.merge(target_data_final, past_feature_data_final, on='ds'))

    future_final = m_final.make_future_dataframe(periods=12, freq='Q')
    future_final = pd.merge(future_final, feature_data_final, on='ds')

    # Getting predictions for 2023
    forecast_final = m_final.predict(future_final)
    df_final = pd.merge(forecast_final[['ds','yhat','yhat_lower','yhat_upper']], data_city_target, on='ds')
    df_final['city'] = city
    df_final = df_final.rename(columns={'ds': 'date'})
    
    # Adding predictions to an overall dataframe
    total_predictions = pd.concat([total_predictions,df_final])
    
    # Saving model in a pickle file
    model_filename = f'{location}_model_{city}.pkl'
    pickle.dump(m_final, open(model_filename, 'wb'))

# Saving overall dataframe
total_predictions.to_csv(f'{location}_total_predictions.csv', index=False)

# Location summary dataframe
location_summary = pd.DataFrame({'city':cities_list,'error_margin':error_list,'number_of_features':num_feats})
location_summary.to_csv(f'{location}_summary.csv', index=False)

# Features summary dataframe
features_summary = pd.DataFrame({'city':cities_TOTAL,'feature':features_model_TOTAL})
features_summary['feature'] = features_summary['feature'].str.replace('_1yr_before', '')
features_summary.to_csv(f'{location}_features_summary.csv', index=False)

--- 1 Aberdeen
2 features selected for Aberdeen with error margin of 0.06 % : ['Oranges (1kg)_1yr_before', 'Meal, Inexpensive Restaurant_1yr_before']
--- 2 Amsterdam
3 features selected for Amsterdam with error margin of 0.14 % : ['Domestic Beer (0.5 liter bottle)_1yr_before', 'Apartment (3 bedrooms) Outside of Centre_1yr_before', 'mortgage_as_percentage_of_income_1yr_before']
--- 3 Athens
3 features selected for Athens with error margin of 0.16 % : ['Apartment (1 bedroom) Outside of Centre_1yr_before', 'time_index_1yr_before', 'Taxi 1hour Waiting (Normal Tariff)_1yr_before']
--- 4 Banja Luka
3 features selected for Banja Luka with error margin of 0.63 % : ['Taxi 1hour Waiting (Normal Tariff)_1yr_before', 'mortgage_as_percentage_of_income_1yr_before', 'pollution_index_1yr_before']
--- 5 Barcelona
3 features selected for Barcelona with error margin of 0.42 % : ['healthcare_index_1yr_before', 'Onion (1kg)_1yr_before', 'Water (0.33 liter bottle) _1yr_before']
--- 6 Belgrade
5 features sel

--- 40 Kaunas
3 features selected for Kaunas with error margin of 0.14 % : ['Milk (regular), (1 liter)_1yr_before', 'Apartment (3 bedrooms) Outside of Centre_1yr_before', 'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)_1yr_before']
--- 41 Kazan
7 features selected for Kazan with error margin of 1.04 % : ['Loaf of Fresh White Bread (500g)_1yr_before', 'Potato (1kg)_1yr_before', 'rent_index_1yr_before', 'exp_pollution_index_1yr_before', 'quality_of_life_index_1yr_before', 'traffic_time_index_1yr_before', 'time_index_1yr_before']
--- 42 Kharkiv
4 features selected for Kharkiv with error margin of 0.14 % : ['Apples (1kg)_1yr_before', 'Oranges (1kg)_1yr_before', 'climate_index_1yr_before', 'Eggs (regular) (12)_1yr_before']
--- 43 Kiev (Kyiv)
7 features selected for Kiev (Kyiv) with error margin of 0.19 % : ['Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)_1yr_before', 'Apartment (1 bedroom) in City Centre_1yr_before', 'International Primary School, Yearly for 1 Child_1yr_befo

--- 66 Oslo
7 features selected for Oslo with error margin of 0.2 % : ['safety_index_1yr_before', 'healthcare_index_1yr_before', 'Fitness Club, Monthly Fee for 1 Adult_1yr_before', 'Apartment (1 bedroom) Outside of Centre_1yr_before', 'crime_index_1yr_before', 'Domestic Beer (0.5 liter draught)_1yr_before', 'price_to_rent_ratio_city_centre_1yr_before']
--- 67 Ostrava
0 features selected for Ostrava with error margin of 0 % : []
--- 68 Paris
3 features selected for Paris with error margin of 0.35 % : ['Banana (1kg)_1yr_before', 'Taxi Start (Normal Tariff)_1yr_before', 'Oranges (1kg)_1yr_before']
--- 69 Perm
5 features selected for Perm with error margin of 1.01 % : ['Chicken Fillets (1kg)_1yr_before', 'Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment_1yr_before', 'Eggs (regular) (12)_1yr_before', 'Apartment (1 bedroom) in City Centre_1yr_before', 'Onion (1kg)_1yr_before']
--- 70 Plovdiv
4 features selected for Plovdiv with error margin of 0.72 % : ['Onion (1kg)_1

--- 96 Timisoara
4 features selected for Timisoara with error margin of 38.52 % : ['Taxi 1hour Waiting (Normal Tariff)_1yr_before', 'purchasing_power_incl_rent_index_1yr_before', 'pollution_index_1yr_before', 'gross_rental_yield_outside_of_centre_1yr_before']
--- 97 Tirana
5 features selected for Tirana with error margin of 0.41 % : ['groceries_index_1yr_before', 'Chicken Fillets (1kg)_1yr_before', 'Apartment (1 bedroom) in City Centre_1yr_before', 'quality_of_life_index_1yr_before', 'Water (0.33 liter bottle) _1yr_before']
--- 98 Turin
4 features selected for Turin with error margin of 0.1 % : ['Taxi Start (Normal Tariff)_1yr_before', '1 Pair of Jeans (Levis 501 Or Similar)_1yr_before', 'healthcare_index_1yr_before', 'Average Monthly Net Salary (After Tax)_1yr_before']
--- 99 Tuzla
0 features selected for Tuzla with error margin of 0 % : []
--- 100 Utrecht
3 features selected for Utrecht with error margin of 3.1 % : ['Potato (1kg)_1yr_before', 'International Primary School, Yearly for

In [9]:
# Loading models

cities_list = cities_list # remove for full list
#cities_list.remove('Perm') # model not working
features_model_TOTAL = []
cities_TOTAL = []
num_feats = []
error_list = []
data_5 = data_4.copy()
total_predictions = pd.DataFrame()
target_var = 'Price per Square Meter to Buy Apartment in City Centre'
location = 'centre' # for pickle files

for city in cities_list:
    print('---',cities_list.index(city)+1,city)
    # City data
    data_city = data_5[data_5['city'] == city]
    data_city = data_city[(data_city['date']>='2011-01-01')]
    clean_target_var = target_var.replace(' ', '_').replace('(', '').replace(')', '')
    data_city = data_city.rename(columns={'date': 'ds'})
    
    # Target city data
    data_city_target = data_city[['ds',target_var]].reset_index(drop=True)
    data_city_target.columns = ['ds','y']
    
    # Features city data
    cols_to_remove = ['city','country','year','Price per Square Meter to Buy Apartment in City Centre',
                     'Price per Square Meter to Buy Apartment Outside of Centre']
    data_city_features = data_city.drop(columns=cols_to_remove)
    
    # Loading model from a pickle file
    model_filename = f'{location}_model_{city}.pkl'
    m_final = pickle.load(open(model_filename, 'rb'))
    features_model = list(m_final.extra_regressors.keys()) # getting features from saved model

    # Final city model and preditions for 2023
    feature_data_final = data_city_features[['ds'] + features_model]
    target_data_final = data_city_target[data_city_target['ds']<='2020-12-31']
    past_feature_data_final = feature_data_final[feature_data_final['ds']<='2020-12-31']
    future_feature_data_final = feature_data_final[feature_data_final['ds']>'2020-12-31']

    future_final = m_final.make_future_dataframe(periods=12, freq='Q')
    future_final = pd.merge(future_final, feature_data_final, on='ds')

    # Getting predictions for 2023
    forecast_final = m_final.predict(future_final)
    df_final = pd.merge(forecast_final[['ds','yhat','yhat_lower','yhat_upper']], data_city_target, on='ds')
    df_final['city'] = city
    df_final = df_final.rename(columns={'ds': 'date'})
    
    # Adding predictions to an overall dataframe
    total_predictions = pd.concat([total_predictions,df_final])
    
    for feat in features_model:
        features_model_TOTAL.append(feat) # adding best feature to list of variables for all cities models
        cities_TOTAL.append(city) # adding city to create dataframe for feats list
    
    # Creating dataframe with model's overall absolute variation
    df_temp = df_final[['date','yhat','y']].tail(8) # 2021 and 2022 (8 quarters)
    df_temp['abs_var'] = abs((df_temp['yhat']-df_temp['y'])/df_temp['y']) 
    overall_min = df_temp['abs_var'].sum()/8
    error_list.append(overall_min)
    num_feats.append(len(features_model))

    
# Saving overall dataframe
total_predictions.to_csv(f'{location}_total_predictions.csv', index=False)

# Location summary dataframe
location_summary = pd.DataFrame({'city':cities_list,'error_margin':error_list,'number_of_features':num_feats})
location_summary.to_csv(f'{location}_summary.csv', index=False)

# Features summary dataframe
features_summary = pd.DataFrame({'city':cities_TOTAL,'feature':features_model_TOTAL})
features_summary['feature'] = features_summary['feature'].str.replace('_1yr_before', '')
features_summary.to_csv(f'{location}_features_summary.csv', index=False)

--- 1 Aberdeen
--- 2 Amsterdam
--- 3 Athens
--- 4 Banja Luka
--- 5 Barcelona
--- 6 Belgrade
--- 7 Berlin
--- 8 Bialystok
--- 9 Bologna
--- 10 Bolzano-Bozen
--- 11 Brasov
--- 12 Bratislava
--- 13 Brno
--- 14 Brussels
--- 15 Bucharest
--- 16 Budapest
--- 17 Catania
--- 18 Chisinau
--- 19 Cluj-Napoca
--- 20 Constanta
--- 21 Copenhagen
--- 22 Coventry
--- 23 Debrecen
--- 24 Dnipro
--- 25 Dublin
--- 26 Eindhoven
--- 27 Espoo
--- 28 Florence
--- 29 Frankfurt
--- 30 Gdansk
--- 31 Gdynia
--- 32 Geneva
--- 33 Groningen
--- 34 Haarlem
--- 35 Helsinki
--- 36 Iasi
--- 37 Katowice
--- 38 Kaunas
--- 39 Kharkiv
--- 40 Kiev (Kyiv)
--- 41 Kosice
--- 42 Krakow (Cracow)
--- 43 Lisbon
--- 44 Liverpool
--- 45 Ljubljana
--- 46 Lodz
--- 47 London
--- 48 Luxembourg
--- 49 Lviv
--- 50 Lyon
--- 51 Madrid
--- 52 Manchester
--- 53 Milan
--- 54 Minsk
--- 55 Moscow
--- 56 Munich
--- 57 Nis
--- 58 Novi Sad
--- 59 Odessa (Odesa)
--- 60 Oradea
--- 61 Oslo
--- 62 Ostrava
--- 63 Paris
--- 64 Plovdiv
--- 65 Plzen
--- 66 

In [10]:
total_predictions

,date,yhat,yhat_lower,yhat_upper,y,city
0,2011-03-31,3752.213621,3526.496198,3972.761633,3676.020096,Aberdeen
1,2011-06-30,3752.549030,3523.675399,3956.180411,3676.020096,Aberdeen
2,2011-09-30,3752.716992,3531.903031,3969.330128,3676.020096,Aberdeen
3,2011-12-31,3752.554314,3511.761086,3968.155905,3676.020096,Aberdeen
4,2012-03-31,3758.582529,3522.819241,3984.160483,3676.020096,Aberdeen
...,...,...,...,...,...,...
47,2022-12-31,18731.797628,15726.737356,21793.864658,18743.129477,Zurich
48,2023-03-31,15057.913103,12300.142773,18192.308309,NaN,Zurich
49,2023-06-30,15135.730103,12057.673870,18303.352255,NaN,Zurich
50,2023-09-30,14180.902500,11057.632671,17089.474060,NaN,Zurich


In [11]:
#location_summary = pd.read_csv('centre_summary.csv')
location_summary

,city,error_margin,number_of_features
0,Aberdeen,0.000434,2
1,Amsterdam,0.000277,3
2,Athens,0.001117,3
3,Banja Luka,0.001382,3
4,Barcelona,0.002300,3
...,...,...,...
95,Vilnius,0.006078,5
96,Warsaw,0.001432,7
97,Wroclaw,0.002505,3
98,Zagreb,0.003390,10


In [12]:
features_summary.head(10)

,city,feature
0,Aberdeen,Oranges (1kg)
1,Aberdeen,"Meal, Inexpensive Restaurant"
2,Amsterdam,Domestic Beer (0.5 liter bottle)
3,Amsterdam,Apartment (3 bedrooms) Outside of Centre
4,Amsterdam,mortgage_as_percentage_of_income
5,Athens,Apartment (1 bedroom) Outside of Centre
6,Athens,time_index
7,Athens,Taxi 1hour Waiting (Normal Tariff)
8,Banja Luka,Taxi 1hour Waiting (Normal Tariff)
9,Banja Luka,mortgage_as_percentage_of_income


In [13]:
data_tableau = pd.read_csv('data_for_tableau.csv')
data_tableau['date'] = pd.to_datetime(data_tableau['date'])
data_tableau

,city,date,country,year,1 Pair of Jeans (Levis 501 Or Similar),1 Pair of Men Leather Business Shoes,1 Pair of Nike Running Shoes (Mid-Range),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",Apartment (1 bedroom) Outside of Centre,Apartment (1 bedroom) in City Centre,...,exp_pollution_index,traffic_time_index,quality_of_life_index,healthcare_index,climate_index,cpi_index,house_price_to_income_ratio,pollution_index,purchasing_power_incl_rent_index,safety_index
0,Aberdeen,2010-03-31,United Kingdom,2010Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,50.103774,31.946650,132.925432,64.416185,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083
1,Aberdeen,2010-06-30,United Kingdom,2010Q2,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,50.103774,31.946650,132.925432,64.416185,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083
2,Aberdeen,2010-09-30,United Kingdom,2010Q3,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,50.103774,31.946650,132.925432,64.416185,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083
3,Aberdeen,2010-12-31,United Kingdom,2010Q4,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,50.103774,31.946650,132.925432,64.416185,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083
4,Aberdeen,2011-03-31,United Kingdom,2011Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,50.103774,31.946650,132.925432,64.416185,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6099,Zurich,2022-12-31,Switzerland,2022Q4,94.265796,158.677770,106.578012,50.438374,1503.576144,1961.96247,...,28.889259,32.947826,200.891758,74.207377,81.481962,131.238266,7.936734,17.915810,129.792429,83.219057
6100,Zurich,2023-03-31,Switzerland,2023Q1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6101,Zurich,2023-06-30,Switzerland,2023Q2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6102,Zurich,2023-09-30,Switzerland,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Moscow and Saint Petersburg have abnormally high prices in 2012, Prague in 2011 and Brno in 2010 and 2011.
# Replacing with the average: 

for col in ['Price per Square Meter to Buy Apartment in City Centre','Price per Square Meter to Buy Apartment Outside of Centre']:
    for city in ['Moscow', 'Saint Petersburg']:
        average_2011_2013 = data_tableau[(data_tableau['city'] == city) & 
                                           ((data_tableau['date'].dt.year == 2011) | (data_tableau['date'].dt.year == 2013))][col]

        average_value = average_2011_2013.mean()

        data_tableau.loc[(data_tableau['city'] == city) & (data_tableau['date'].dt.year == 2012),
                           col] = average_value

    for city in ['Prague']:
        average_2010_2012 = data_tableau[(data_tableau['city'] == city) & 
                                           ((data_tableau['date'].dt.year == 2010) | (data_tableau['date'].dt.year == 2012))][col]

        average_value = average_2010_2012.mean()

        data_tableau.loc[(data_tableau['city'] == city) & (data_tableau['date'].dt.year == 2011),
                           col] = average_value
        
    for city in ['Brno']:
        average_2012 = data_tableau[(data_tableau['city'] == city) & (data_tableau['date'].dt.year == 2012)][col]

        average_value = average_2012.mean()

        data_tableau.loc[(data_tableau['city'] == city) & (data_tableau['date'].dt.year <= 2011),
                           col] = average_value

In [15]:
data_tableau['Price per Square Meter to Buy Apartment on Average'] = (
    data_tableau['Price per Square Meter to Buy Apartment in City Centre'] + 
    data_tableau['Price per Square Meter to Buy Apartment Outside of Centre']) / 2

In [16]:
data_tableau[['Price per Square Meter to Buy Apartment on Average',
              'Price per Square Meter to Buy Apartment in City Centre',
              'Price per Square Meter to Buy Apartment Outside of Centre']]

,Price per Square Meter to Buy Apartment on Average,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre
0,2603.847568,3676.020096,1531.67504
1,2603.847568,3676.020096,1531.67504
2,2603.847568,3676.020096,1531.67504
3,2603.847568,3676.020096,1531.67504
4,2603.847568,3676.020096,1531.67504
...,...,...,...
6099,14880.856909,18743.129477,11018.58434
6100,NaN,NaN,NaN
6101,NaN,NaN,NaN
6102,NaN,NaN,NaN


In [17]:
data_tableau_2 = pd.merge(data_tableau, total_predictions[['city','date','yhat']], on=['city','date'], how='left')
data_tableau_2 = data_tableau_2.rename(columns={'yhat': 'price_predictions_centre'})
data_tableau_2[['city','date','Price per Square Meter to Buy Apartment in City Centre','price_predictions_centre']]

,city,date,Price per Square Meter to Buy Apartment in City Centre,price_predictions_centre
0,Aberdeen,2010-03-31,3676.020096,NaN
1,Aberdeen,2010-06-30,3676.020096,NaN
2,Aberdeen,2010-09-30,3676.020096,NaN
3,Aberdeen,2010-12-31,3676.020096,NaN
4,Aberdeen,2011-03-31,3676.020096,3752.213621
...,...,...,...,...
6099,Zurich,2022-12-31,18743.129477,18731.797628
6100,Zurich,2023-03-31,NaN,15057.913103
6101,Zurich,2023-06-30,NaN,15135.730103
6102,Zurich,2023-09-30,NaN,14180.902500


In [18]:
data_tableau_2['price_predictions_outside'] = (data_tableau_2['Price per Square Meter to Buy Apartment Outside of Centre']*
                                               data_tableau_2['price_predictions_centre']/
                                               data_tableau_2['Price per Square Meter to Buy Apartment in City Centre'])

data_tableau_2['price_predictions_average'] = (
    data_tableau_2['price_predictions_centre'] + 
    data_tableau_2['price_predictions_outside']) / 2

data_tableau_2

,city,date,country,year,1 Pair of Jeans (Levis 501 Or Similar),1 Pair of Men Leather Business Shoes,1 Pair of Nike Running Shoes (Mid-Range),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",Apartment (1 bedroom) Outside of Centre,Apartment (1 bedroom) in City Centre,...,climate_index,cpi_index,house_price_to_income_ratio,pollution_index,purchasing_power_incl_rent_index,safety_index,Price per Square Meter to Buy Apartment on Average,price_predictions_centre,price_predictions_outside,price_predictions_average
0,Aberdeen,2010-03-31,United Kingdom,2010Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
1,Aberdeen,2010-06-30,United Kingdom,2010Q2,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
2,Aberdeen,2010-09-30,United Kingdom,2010Q3,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
3,Aberdeen,2010-12-31,United Kingdom,2010Q4,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
4,Aberdeen,2011-03-31,United Kingdom,2011Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,3752.213621,1563.422342,2657.817982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6099,Zurich,2022-12-31,Switzerland,2022Q4,94.265796,158.677770,106.578012,50.438374,1503.576144,1961.96247,...,81.481962,131.238266,7.936734,17.915810,129.792429,83.219057,14880.856909,18731.797628,11011.922649,14871.860138
6100,Zurich,2023-03-31,Switzerland,2023Q1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15057.913103,NaN,NaN
6101,Zurich,2023-06-30,Switzerland,2023Q2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15135.730103,NaN,NaN
6102,Zurich,2023-09-30,Switzerland,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14180.902500,NaN,NaN


In [19]:
data_tableau_2[data_tableau_2['city']=='Moscow'][['date','Price per Square Meter to Buy Apartment in City Centre']].head(16)

,date,Price per Square Meter to Buy Apartment in City Centre
3248,2010-03-31,5237.138658
3249,2010-06-30,5237.138658
3250,2010-09-30,5237.138658
3251,2010-12-31,5237.138658
3252,2011-03-31,7916.006225
3253,2011-06-30,7916.006225
3254,2011-09-30,7916.006225
3255,2011-12-31,7916.006225
3256,2012-03-31,6605.571983
3257,2012-06-30,6605.571983


In [22]:
for city in russian_cities:
    data_tableau_2 = data_tableau_2[data_tableau_2['city']!=city] # removing russian cities
data_tableau_2

,city,date,country,year,1 Pair of Jeans (Levis 501 Or Similar),1 Pair of Men Leather Business Shoes,1 Pair of Nike Running Shoes (Mid-Range),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",Apartment (1 bedroom) Outside of Centre,Apartment (1 bedroom) in City Centre,...,climate_index,cpi_index,house_price_to_income_ratio,pollution_index,purchasing_power_incl_rent_index,safety_index,Price per Square Meter to Buy Apartment on Average,price_predictions_centre,price_predictions_outside,price_predictions_average
0,Aberdeen,2010-03-31,United Kingdom,2010Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
1,Aberdeen,2010-06-30,United Kingdom,2010Q2,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
2,Aberdeen,2010-09-30,United Kingdom,2010Q3,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
3,Aberdeen,2010-12-31,United Kingdom,2010Q4,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,NaN,NaN,NaN
4,Aberdeen,2011-03-31,United Kingdom,2011Q1,81.689335,101.090553,91.900502,55.140301,694.359351,785.44296,...,74.169806,109.084082,5.902778,20.344828,119.202522,83.802083,2603.847568,3752.213621,1563.422342,2657.817982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6099,Zurich,2022-12-31,Switzerland,2022Q4,94.265796,158.677770,106.578012,50.438374,1503.576144,1961.96247,...,81.481962,131.238266,7.936734,17.915810,129.792429,83.219057,14880.856909,18731.797628,11011.922649,14871.860138
6100,Zurich,2023-03-31,Switzerland,2023Q1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15057.913103,NaN,NaN
6101,Zurich,2023-06-30,Switzerland,2023Q2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15135.730103,NaN,NaN
6102,Zurich,2023-09-30,Switzerland,2023Q3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14180.902500,NaN,NaN


In [23]:
data_tableau_2.to_csv('data_tableau_2.csv', index=False)

In [31]:
location_summary['number_of_features'].sum()*70

30310

In [35]:
len(data_tableau_2['country'].drop_duplicates())

39